In [1]:
import re
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

C:\Users\angsp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\angsp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\utils\hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:

teacher_path = "./model_final_taskA"
base_student_path = "google/flan-t5-small"
studentRB_path = "./studentRBKD_taskA"
studentFB_path = "./studentFBKD_taskA"
studentRelB_path = "./studentRelBKD_taskA"  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(teacher_path)

teacher_model = AutoModelForSeq2SeqLM.from_pretrained(teacher_path).to(device)
base_student_model = AutoModelForSeq2SeqLM.from_pretrained(base_student_path).to(device)
studentRB_model = AutoModelForSeq2SeqLM.from_pretrained(studentRB_path).to(device)
studentFB_model = AutoModelForSeq2SeqLM.from_pretrained(studentFB_path).to(device)
studentRelB_model = AutoModelForSeq2SeqLM.from_pretrained(studentRelB_path).to(device)


def generate_response(model, sentence):
    prompt = (
    "In exactly 1-2 sentences, identify the specific words or phrases that make the text sarcastic "
    "and explain how they create the sarcastic effect. "
    "Focus only on observable linguistic elements without adding interpretation beyond what's directly evident in the text.\n\n"
    f"Sentence: \"{sentence}\""
)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=2.3,      
        top_p=0.6,         
        top_k=60,
        num_beams=10,          
        no_repeat_ngram_size=3,
        repetition_penalty=1.4, 
        length_penalty=1.0,
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    text = re.sub(r"^(Explanation|Answer|Response)\s*:\s*", "", text, flags=re.IGNORECASE)
    return text


sentences = [
    "Yay my shoe broke!",
    "If the shooter shouldn't have been able to get a gun, the solution is obviously more guns, right? <URL>",
    "Complain to the cellular company for having monopolistic dataplans and ask for an unlimited plan",
    "Wonderful, traffic is even worse than yesterday!",
    "I'm so sorry I can't read sarcasm over the internet",
    "Slept horrible last night / I feel like crap and now I get to go sit in classes all day. Hurray! idontwannagoback",
    "I'm sure a homeless shelter as a permanent address and phone number looks great on a resume to prospective employers.",
    "Trains are delayed on both directions. Instead of seeing people rushing to take the bus or cab, they were taking pictures. Haha..",
    "This guy gets a gold star for such excellent parking in the handicap lot!",
    "Crying before I go into work... This is going to be a great night. #Sarcasm #WishItWasTrue",
    "In a cab on the way home from the airport. What a long day. Work tomorrow is going to be AMAZING. Should be home by 3AM.",
    "hey <user> thanks for making it easy for me to take my music with me . # ihateyourupdates",
    "It could confuse your muscles and make muscle grow in places where you didn't actually work out.",
    "Yay, 2-hour traffic for a 10-minute errand. Exactly what I needed 🙃",
    "How else will we feel superior if not by our amazing taste in phones?",
    "Received a compliment today that I look very relaxed. If only this person knew just how much effort it takes to look this relaxed.",
    "My phone dying at 5% is the highlight of my day.",
    "overweight man repeatedly introduced to overweight woman at party",
    "How dare you type out Obama's name and not praise him you racist",
    "No, it's perfectly safe for nurses to shove pills in your mouth without any education",
    "Great, another inspirational quote on LinkedIn. Just what I needed.",
    "even aside from the blatant misogyny, this is great because we have so much space in our prisons!",
    "DSA4213 is sooo easy even my grandma can score A+"
    "I have never felt more alive than during DSA4213 finetuning, nothing like a few CUDA OOMs to keep the adrenaline going.",
    "I love how DSA4213 keeps me humble, every single assignment and quizzes",
    "DSA4213 is not that hard, I just needed 4 GPUs and divine intervention"
]

for s in sentences:
    teacher_out = generate_response(teacher_model, s)
    base_out = generate_response(base_student_model, s)
    rb_out = generate_response(studentRB_model, s)
    fb_out = generate_response(studentFB_model, s)
    relb_out = generate_response(studentRelB_model, s)

    print(f"\n Sentence: {s}")
    print(f"Teacher (Flan-T5-base):   {teacher_out}")
    print(f"Base Student (T5-small):  {base_out}")
    print(f"RBKD Student:             {rb_out}")
    print(f"FBKD Student:             {fb_out}")
    print(f"RelBKD Student:           {relb_out}")

In [ ]:
teacher_path = "./model_final_taskB"
base_student_path = "./model_final_dualtask"
studentRB_path = "./studentRBKD_taskB"
studentFB_path = "./studentFBKD_taskB"
studentRelB_path = "./studentRelBKD_taskB"   

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(teacher_path)

teacher_model = AutoModelForSeq2SeqLM.from_pretrained(teacher_path).to(device)
base_student_model = AutoModelForSeq2SeqLM.from_pretrained(base_student_path).to(device)
studentRB_model = AutoModelForSeq2SeqLM.from_pretrained(studentRB_path).to(device)
studentFB_model = AutoModelForSeq2SeqLM.from_pretrained(studentFB_path).to(device)
studentRelB_model = AutoModelForSeq2SeqLM.from_pretrained(studentRelB_path).to(device)


def generate_response(model, sentence):
    prompt = (
        "In exactly 1-2 sentences, explain what the speaker actually means by removing the sarcasm "
        "and stating their true intended message directly. "
        "Focus on the genuine sentiment or opinion being expressed beneath the sarcastic language.\n\n"
        f"Sentence: \"{sentence}\"\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.01,      
        top_p=0.6,            
        top_k=60,
        num_beams=10,           
        no_repeat_ngram_size=3,
        repetition_penalty=1.4, 
        length_penalty=1.0,
    )


    text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    text = re.sub(r"^(Explanation|Answer|Response)\s*:\s*", "", text, flags=re.IGNORECASE)
    return text


sentences = [
    "Yay my shoe broke!",
    "If the shooter shouldn't have been able to get a gun, the solution is obviously more guns, right? <URL>",
    "Complain to the cellular company for having monopolistic dataplans and ask for an unlimited plan",
    "Wonderful, traffic is even worse than yesterday!",
    "I'm so sorry I can't read sarcasm over the internet",
    "Slept horrible last night / I feel like crap and now I get to go sit in classes all day. Hurray! idontwannagoback",
    "I'm sure a homeless shelter as a permanent address and phone number looks great on a resume to prospective employers.",
    "Crying before I go into work... This is going to be a great night. #Sarcasm #WishItWasTrue",
    "In a cab on the way home from the airport. What a long day. Work tomorrow is going to be AMAZING. Should be home by 3AM.",
    "hey <user> thanks for making it easy for me to take my music with me . # ihateyourupdates",
    "It could confuse your muscles and make muscle grow in places where you didn't actually work out.",
    "Yay, 2-hour traffic for a 10-minute errand. Exactly what I needed 🙃",
    "This guy gets a gold star for such excellent parking in the handicap lot!",
    "How else will we feel superior if not by our amazing taste in phones?",
    "Received a compliment today that I look very relaxed. If only this person knew just how much effort it takes to look this relaxed.",
    "My phone dying at 5% is the highlight of my day.",
    "overweight man repeatedly introduced to overweight woman at party",
    "How dare you type out Obama's name and not praise him you racist",
    "No, it's perfectly safe for nurses to shove pills in your mouth without any education",
    "Great, another inspirational quote on LinkedIn. Just what I needed.",
    "even aside from the blatant misogyny, this is great because we have so much space in our prisons!",
    "DSA4213 is sooo easy even my grandma can score A+"
    "Trains are delayed on both directions. Instead of seeing people rushing to take the bus or cab, they were taking pictures. Haha..",
    "I have never felt more alive than during DSA4213 finetuning, nothing like a few CUDA OOMs to keep the adrenaline going.",
    "I love how DSA4213 keeps me humble, every single assignment and quizzes",
    "DSA4213 is not that hard, I just needed 4 GPUs and divine intervention"
]


for s in sentences:
    teacher_out = generate_response(teacher_model, s)
    base_out = generate_response(base_student_model, s)
    rb_out = generate_response(studentRB_model, s)
    fb_out = generate_response(studentFB_model, s)
    relb_out = generate_response(studentRelB_model, s)

    print(f"\n Sentence: {s}")
    print(f"Teacher (Flan-T5-base):   {teacher_out}")
    print(f"Base Student (T5-small):  {base_out}")
    print(f"RBKD Student:             {rb_out}")
    print(f"FBKD Student:             {fb_out}")
    print(f"RelBKD Student:           {relb_out}")